In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install ultralytics

In [11]:
# improved_video_annotate.py
import os
import sys
import cv2
from ultralytics import YOLO

# ---------- CONFIG (merged paths from you) ----------
WEIGHTS_PATH = "/content/drive/MyDrive/infosys internship/weights/best.pt"
SOURCE_PATH  = "/content/drive/MyDrive/infosys internship/VIdeos dataset/test"
OUTDIR       = "Output"
IMGSZ        = 896
CONF         = 0.25
# ----------------------------------------------------

def ensure_dir(p):
    os.makedirs(p, exist_ok=True)
    return p

def is_video_file(fn):
    ext = os.path.splitext(fn)[1].lower()
    return ext in {".mp4", ".avi", ".mov", ".mkv", ".MP4", ".AVI", ".MOV", ".MKV"}

def process_single_video(video_path, model, outdir, imgsz, conf):
    print(f"\nProcessing: {video_path}")
    if not os.path.isfile(video_path):
        print(f"  ERROR: file not found: {video_path}")
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"  ERROR: cv2.VideoCapture failed to open {video_path}")
        return

    base = os.path.splitext(os.path.basename(video_path))[0]
    vid_dir = ensure_dir(os.path.join(outdir, "videos_annotated"))
    hit_dir = ensure_dir(os.path.join(outdir, "video_hit_frames", base))

    # read first frame to determine size and fps (and to ensure capture works)
    ok, frame = cap.read()
    if not ok:
        print("  ERROR: failed to read first frame. Empty or corrupted video.")
        cap.release()
        return

    H, W = frame.shape[:2]
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    print(f"  Video properties: width={W}, height={H}, fps={fps}")

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out_path = os.path.join(vid_dir, f"{base}_ann.mp4")
    writer = cv2.VideoWriter(out_path, fourcc, fps, (W, H))

    if not writer.isOpened():
        print(f"  ERROR: VideoWriter failed to open for {out_path}")
        cap.release()
        return
    else:
        print(f"  Writing annotated video to: {out_path}")

    frame_id = 0

    # NOTE: we've already read the first frame, so process it then continue loop
    while True:
        if frame is None:
            break
        frame_id += 1

        # model.predict accepts numpy arrays; keep verbose False
        res = model.predict(frame, imgsz=imgsz, conf=conf, verbose=False)[0]

        # res.plot() returns annotated image array
        ann = res.plot()  # keep as numpy image

        # ensure ann has same size as writer expects; if not, resize
        if ann.shape[1] != W or ann.shape[0] != H:
            ann = cv2.resize(ann, (W, H))

        writer.write(ann)

        # save hit frames if masks or boxes (depending on model output)
        has_hit = False
        if hasattr(res, "masks") and res.masks is not None:
            # masks present
            has_hit = True
        elif len(res.boxes) > 0:
            has_hit = True

        if has_hit:
            hit_fname = os.path.join(hit_dir, f"frame_{frame_id:06d}.jpg")
            cv2.imwrite(hit_fname, ann)

        # read next
        ok, frame = cap.read()
        if not ok:
            break

    cap.release()
    writer.release()
    print(f"  Done processing {base}. Frames processed: ~{frame_id}")

def main():
    # sanity checks
    if not os.path.exists(WEIGHTS_PATH):
        print(f"ERROR: weights path not found: {WEIGHTS_PATH}")
        sys.exit(1)
    if not os.path.exists(SOURCE_PATH):
        print(f"ERROR: source path not found: {SOURCE_PATH}")
        sys.exit(1)

    # load model
    print("Loading YOLO model...")
    model = YOLO(WEIGHTS_PATH)
    print("Model loaded.")

    # if source is a directory -> process all video files inside
    if os.path.isdir(SOURCE_PATH):
        files = sorted(os.listdir(SOURCE_PATH))
        video_files = [os.path.join(SOURCE_PATH, f) for f in files if is_video_file(f)]
        if not video_files:
            print(f"No video files found in directory: {SOURCE_PATH}")
            sys.exit(1)

        print(f"Found {len(video_files)} video(s) in directory. Starting batch processing...")
        for vid in video_files:
            process_single_video(vid, model, OUTDIR, IMGSZ, CONF)
    else:
        # assume it's a single video file
        process_single_video(SOURCE_PATH, model, OUTDIR, IMGSZ, CONF)

    print("\nAll done. Check the outputs/ folder (or the printed paths).")

if __name__ == "__main__":
    main()


Loading YOLO model...
Model loaded.
Found 4 video(s) in directory. Starting batch processing...

Processing: /content/drive/MyDrive/infosys internship/VIdeos dataset/test/10394114-uhd_4096_2160_30fps.mp4
  Video properties: width=4096, height=2160, fps=29.97002997002997
  Writing annotated video to: Output/videos_annotated/10394114-uhd_4096_2160_30fps_ann.mp4
  Done processing 10394114-uhd_4096_2160_30fps. Frames processed: ~1784

Processing: /content/drive/MyDrive/infosys internship/VIdeos dataset/test/Screen Recording 2025-12-04 184349.mp4
  Video properties: width=1862, height=920, fps=30.0
  Writing annotated video to: Output/videos_annotated/Screen Recording 2025-12-04 184349_ann.mp4
  Done processing Screen Recording 2025-12-04 184349. Frames processed: ~393

Processing: /content/drive/MyDrive/infosys internship/VIdeos dataset/test/drone_footage_oil_spill_original.mp4
  Video properties: width=1280, height=720, fps=30.0
  Writing annotated video to: Output/videos_annotated/drone_

In [15]:
import shutil
import os

output_dir = '/content/Output'
zip_file_name = '/content/Output.zip'

# Create a zip archive of the output directory
shutil.make_archive(os.path.splitext(zip_file_name)[0], 'zip', output_dir)

print(f"The folder '{output_dir}' has been successfully zipped to '{zip_file_name}'")

The folder '/content/Output' has been successfully zipped to '/content/Output.zip'


In [16]:
from google.colab import files
files.download('/content/Output.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>